In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 2.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [1]:
import pickle
img_caption_file_name = '/content/drive/MyDrive/mscoco_40k_clustered.pkl'

with open(img_caption_file_name, 'rb') as f:
  df_mscoco = pickle.load(f)

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)
model_id = "microsoft/Phi-3-medium-4k-instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

messages = [
    {"role": "user", "content": "Can you provide ways to eat combinations of bananas and dragonfruits?"},
    {"role": "assistant", "content": "Sure! Here are some ways to eat bananas and dragonfruits together: 1. Banana and dragonfruit smoothie: Blend bananas and dragonfruits together with some milk and honey. 2. Banana and dragonfruit salad: Mix sliced bananas and dragonfruits together with some lemon juice and honey."},
    {"role": "user", "content": "What about solving an 2x + 3 = 7 equation?"},
]

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

output = pipe(messages, **generation_args)
print(output[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


 To solve the equation 2x + 3 = 7, you need to isolate the variable x. Here's how you can do it step by step:

1. Subtract 3 from both sides of the equation:
   2x + 3 - 3 = 7 - 3
   2x = 4

2. Divide both sides of the equation by 2:
   2x / 2 = 4 / 2
   x = 2

So, the solution to the equation 2x + 3 = 7 is x = 2.


In [43]:
def decompose_sentence(sentence):
    messages = [
            {"role": "user", "content": f"Example 1:\nOriginal: An institutional looking room holds an exit sign, chairs, recording equipment and a man with one arm aloft that appears to be demonstrating something with a device in the held-up hand for the benefit of a woman behind him that is watching closely.\n\nDecomposed form: \"An exit sign.\"|\"Chairs.\"|\"Recording equipment.\"|\"A man with one arm aloft appears to be demonstrating with a device in the held-up hand.\"|\"A woman behind a man is watching him closely.\"\n\nExample 2:\nOriginal: Black night, a blurry shot shows a tall fence that angles, in a kind of Z shape, around what appears to be a parking area with a large building fronting it, as a kid with a red helmet takes a fierce swing in the batting area, inside the fence.\n\nDecomposed form: \"In a black night, a blurry shot shows a tall fence that angles, in a kind of Z shape.\"|\"Around appears to be a parking area with a large building fronting it.\"|\"A kid with a red helmet takes a fierce swing in the batting area, inside the fence.\"\n\nExample 3:\nOriginal: An older looking model computer components; two disk drive boxes stacked, a keyboard unit, and a more modern looking computer monitor with apple logo center bottom, and on screen at top centered in light blue font letters is printed \"APPLE II\"\n\nDecomposed form: \"An older looking model computer components.\"|\"Computer disk drive boxes stacked.\"|\"A keyboard unit.\"|\"A modern computer monitor with apple logo at the center bottom.\"|\"On the screen of a modern computer monitor, at the top center, light blue font letters print the words APPLE II.\"\n\nExample 4:\nOriginal: A motel-like room features all of its furniture in rows, including a sofa, double bed, stuffed chair, desk chair, desk, bureau and television, all of which, like the one piece of artwork, carpet, linen and window drapes, are all in neutrals.\n\nDecomposed form: \"A motel room.\"|\"A neutral colored sofa.\"|\"A neutral colored double bed.\"|\"A neutral colored stuffed chair.\"|\"A desk chair.\"|\"A desk.\"|\"A bureau.\"|\"A television.\"|\"One piece of artwork.\"|\"Neutral colored linen.\"|\"Neutral colored window drapes.\"\n\nExample 5:\nOriginal: A room shows a twin bed, crib, double bed, and a round bed, all featuring checks and pillows, either white, or checked, high windows, a brick wall, a piece of art against a white wall, and a separate vertical surface with framed items on it.\n\nDecomposed form: \"A room.\"|\"A twin bed featuring checks and pillows, either white, or checked.\"|\"A crib.\"|\"A double bed featuring checks and pillows, either white, or checked.\"|\"A round bed featuring checks and pillows, either white, or checked.\"|\"High windows.\"|\"A brick wall.\"|\"A piece of art against a white wall.\"|\"A vertical surface with framed items on it.\"\n\nExample 6:\nOriginal: A street view shows a building entrance and traffic, including a bus, in the background, and to the foreground, a black iron fence, and a bench with an older, seated person, looking at a flock of pigeons scattered on the ground and bench.\n\nDecomposed form: \"A street view shows a building entrance.\"|\"A street view shows traffic, including a bus in the background.\"|\"In the foreground is a black iron fence.\"|\"In the foreground is a bench with an older person sitting on it\"|\"A flock of pigeons scattered on the ground and bench.\"\n\nExample 7:\nOriginal: Off in the distance, a few people dot the stands and the fenced in area bordering a ball field, while an umpire and three uniformed baseball players, including a batter, catcher and outfielder, strike playing poses in the foreground.\n\nDecomposed form: \"A few people dot the stands bordering a ball field.\"|\"A few people dot the fenced-in area bordering a ball field.\"|\"A baseball umpire.\"|\"A baseball batter.\"|\"A baseball catcher.\"|\"An outfielder.\"\n\nExample 8:\nOriginal: A counter top with a plate with a fork and few scraps of food and a teddy bear lying on side with arm outstretched on plate near fork, with another plate with an apple and two bowls with produce, a canister and some metal objects.\n\nDecomposed form: \"A counter top.\"|\"A plate with a fork and few scraps of food.\"|\"A teddy bear lying with arm outstretched on plate near fork.\"|\"A plate with an apple.\"|\"Bowls with apples.\"|\"A canister.\"|\"Metal objects.\"\n\nExample 9:\nOriginal: Indirect lighting in a darkened room plays across a table covered with the remains of a meal, thereby giving everything, including the table, a basket, plates with uneaten food, utensils and glasses with liquid, a reddish patina.\n\nDecomposed form: \"Indirect lighting in a darkened room shines across a table covered with the remains of a meal.\"|\"The table has a reddish patina.\"|\"The basket has a reddish patina.\"|\"The plates with uneaten food have a reddish patina.\"|\"Utensils have a reddish patina.\"|\"Glasses with liquid have a reddish patina.\"\n\nExample 10:\nOriginal: the long view of a parking lot shows a row of parked vehicles, including a bus, which appears to be the destination of a number of similarly dressed women with luggage that are moving towards it from the immediate foreground.\n\nDecomposed form: \"A parking lot shows a row of parked vehicles.\"|\"A bus.\"|\"Similarly dressed women with luggages.\"\n\nExample 11:\nOriginal: Corner view of a table with plaid cloth, red and white checkerboard napkins, a pizza, a container of green leafy vegetable, and some bottles, with the hips of a person wearing an apron standing beside table in green grass.\n\nDecomposed form: \"Corner view of a table with plaid cloth.\"|\"Red and white checkerboard napkins.\"|\"A pizza.\"|\"A container of green leafy vegetables.\"|\"Some bottles.\"|\"The hips of a person wearing an apron standing beside a table in green grass.\"\n\nExample 12:\nOriginal: A room with natural, unfinished, pale wood walls and built in shelves, also has curtained sliding doors that open onto a patio with a lounge chair, a big double bed with a blue blanket, and various framed photos and books.\n\nDecomposed form: \"A room.\"|\"Natural, unfinished, pale wood walls.\"|\"Built-in shelves.\"|\"Curtained sliding doors.\"|\"Sliding doors that open onto a patio with a lounge chair.\"|\"A big double bed with a blue blanket.\"|\"Framed photos.\"|\"Books.\"\n\nExample 13:\nOriginal: A minimally furnished room shows small window pairs with blinds and no curtains, an unmade double bed with a heavy, taupe duvet, and a white border near the ceiling with colorful birds trailing in both directions.\n\nDecomposed form: \"A minimally furnished room.\"|\"Small window pairs with blinds and no curtains.\"|\"An unmade double bed with a heavy, taupe duvet.\"|\"A white border near the ceiling with colorful birds trailing in both directions.\"\n\nExample 14:\nOriginal: A blurry night-time street scene shows a higher and lower roadway with overpasses and lots of traffic going the same way, while to the left an area of red glare with an arrow and number, suggests a caution area.\n\nDecomposed form: \"A blurry night-time street scene shows a higher and lower roadway with overpasses and lots of traffic going the same way.\"|\"An area of red glare with an arrow and number, suggests a caution area.\"\n\nExample 15:\nOriginal: Several people are standing, or sitting, in the vicinity of a scoreboard, to the front of which lies a tennis court, featuring a player with his knees bent, who is also grasping a racquet with both hands.\n\nDecomposed form: \"Several people are standing, or sitting, in the vicinity of a scoreboard.\"|\"In front of the scoreboard lies a tennis court.\"|\"A tennis player with his knees bent, who is grasping a racquet with both hands.\"\n\nExample 16:\nOriginal: Black and white of a blacksmith interior with a horse, a man standing with arm out to it, and a man ducking under it's head and holding a long metal file in both hands, with horses on poles on the wall.\n\nDecomposed form: \"Black and white photo of a blacksmith interior with a horse.\"|\"Black and white photo of a man standing with arm out to a horse.\"|\"Black and white photo of a man ducking under a horses head and holding a long metal file in both hands.\"|\"Black and white photo with horse poles on the wall.\"\n\nExample 17:\nOriginal: A blurry living room scene suggests a wide screen TV with a game playing, bookshelves, a low cluttered table, a couch, and closest and clearest, a sports t-shirt with the number thirty-five on it.\n\nDecomposed form: \"A blurry living room scene.\"|\"A wide screen TV with a sports game playing.\"|\"Bookshelves.\"|\"A low cluttered table.\"|\"A couch.\"|\"A sports t-shirt with the number 35 on it.\"\n\nExample 18:\nOriginal: In the first picture a boy is flipping his skateboard, in the second picture is skating and lifting the board slightly off the ground, and in the third he is jumping into the air onto the skateboard.\n\nDecomposed form: \"A boy is flipping his skateboard.\"|\"A boy is skating and lifting the skateboard slightly off the ground.\"|\"A boy is jumping into the air onto the skateboard.\"\n\nExample 19:\nOriginal: A table with a person holding a sandwich in hand over a plate with a knife on it, and another plate with a knife, napkin, and french bread sandwich, along with a bottle and a glass with beverages.\n\nDecomposed form: \"A table with a person holding a sandwich in hand.\"|\"A sandwich in hand over a plate with a knife on it.\"|\"A plate with a knife, napkin, and french bread sandwich.\"|\"A bottle.\"|\"A glass with beverages.\"\n\nExample 20:\nOriginal: An impressive array of motorcycles recedes into the distance between a driveway and a building fronted with cars and a grassy area, as lots of people mill about the driveway, around the bikes.\n\nDecomposed form: \"An array of motorcycles between a driveway and a building.\"|\"A building fronted with cars and a grassy area.\"|\"Lots of people mill about the driveway, around the bikes.\"\n\nGiven these examples of decomposing original sentences into their decomposed forms, decompose the given sentence:\n{sentence}\nPlease response in the same format, outputing only the decomposed sentences."}
    ]
    output = pipe(messages, **generation_args)
    print(output[0]['generated_text'])
    return output[0]['generated_text']

In [37]:
import time
import pandas as pd
import pickle

class UnionFind:
    def __init__(self, size):
        self.parent = list(range(size))
        self.rank = [1] * size

    def find(self, p):
        if self.parent[p] != p:
            self.parent[p] = self.find(self.parent[p])  # Path compression
        return self.parent[p]

    def union(self, p, q):
        rootP = self.find(p)
        rootQ = self.find(q)
        if rootP != rootQ:
            # Union by rank
            if self.rank[rootP] > self.rank[rootQ]:
                self.parent[rootQ] = rootP
            elif self.rank[rootP] < self.rank[rootQ]:
                self.parent[rootP] = rootQ
            else:
                self.parent[rootQ] = rootP
                self.rank[rootP] += 1

def check_dependency(sentence, segment1, segment2):
    messages = [
          {"role": "system", "content": "Example 1:\nOverall sentence: An institutional looking room holds an exit sign, chairs, recording equipment and a man with one arm aloft that appears to be demonstrating something with a device in the held-up hand for the benefit of a woman behind him that is watching closely.\nDecomposed sentence: [\"An exit sign.\"]|[\"Chairs.\"]|[\"Recording equipment.\"]|[\"A man with one arm aloft appears to be demonstrating with a device in the held-up hand.\",\"A woman behind a man is watching him closely.\"]\nExample 2:\nOverall sentence: Black night, a blurry shot shows a tall fence that angles, in a kind of Z shape, around what appears to be a parking area with a large building fronting it, as a kid with a red helmet takes a fierce swing in the batting area, inside the fence.\nDecomposed sentence: [\"In a black night, a blurry shot shows a tall fence that angles, in a kind of Z shape.\",\"Around appears to be a parking area with a large building fronting it.\",\"A kid with a red helmet takes a fierce swing in the batting area, inside the fence.\"]\nExample 3:\nOverall sentence: An older looking model computer components; two disk drive boxes stacked, a keyboard unit, and a more modern looking computer monitor with apple logo center bottom, and on screen at top centered in light blue font letters is printed APPLE II\nDecomposed sentence: [\"An older looking model computer components.\",\"Computer disk drive boxes stacked.\",\"A keyboard unit.\"]|[\"A modern computer monitor with apple logo at the center bottom.\",\"On the screen of a modern computer monitor, at the top center, light blue font letters print the words APPLE II.\"]\nExample 4:\nOverall sentence: A motel-like room features all of its furniture in rows, including a sofa, double bed, stuffed chair, desk chair, desk, bureau and television, all of which, like the one piece of artwork, carpet, linen and window drapes, are all in neutrals.\nDecomposed sentence: [\"A motel room.\"]|[\"A neutral colored sofa.\"]|[\"A neutral colored double bed.\"]|[\"A neutral colored stuffed chair.\"]|[\"A desk chair.\",\"A desk.\"]|[\"A bureau.\"]|[\"A television.\"]|[\"One piece of artwork.\"]|[\"Neutral colored linen.\"]|[\"Neutral colored window drapes.\"]\nExample 5:\nOverall sentence: A room shows a twin bed, crib, double bed, and a round bed, all featuring checks and pillows, either white, or checked, high windows, a brick wall, a piece of art against a white wall, and a separate vertical surface with framed items on it.\nDecomposed sentence: [\"A room.\"]|[\"A twin bed featuring checks and pillows, either white, or checked.\"]|[\"A crib.\"]|[\"A double bed featuring checks and pillows, either white, or checked.\"]|[\"A round bed featuring checks and pillows, either white, or checked.\"]|[\"High windows.\"]|[\"A brick wall.\"]|[\"A piece of art against a white wall.\"]|[\"A vertical surface with framed items on it.\"]\nExample 6:\nOverall sentence: A street view shows a building entrance and traffic, including a bus, in the background, and to the foreground, a black iron fence, and a bench with an older, seated person, looking at a flock of pigeons scattered on the ground and bench.\nDecomposed sentence: [\"A street view shows a building entrance.\",\"A street view shows traffic, including a bus in the background.\"]|[\"In the foreground is a black iron fence.\"]|[\"In the foreground is a bench with an older person sitting on it\",\"A flock of pigeons scattered on the ground and bench.\"]\nExample 7:\nOverall sentence: Off in the distance, a few people dot the stands and the fenced in area bordering a ball field, while an umpire and three uniformed baseball players, including a batter, catcher and outfielder, strike playing poses in the foreground.\nDecomposed form: [\"A few people dot the stands bordering a ball field.\"]|[\"A few people dot the fenced-in area bordering a ball field.\"]|[\"A baseball umpire.\",\"A baseball batter.\",\"A baseball catcher.\",\"An outfielder.\"]\nExample 8:\nOverall sentence: A counter top with a plate with a fork and few scraps of food and a teddy bear lying on side with arm outstretched on plate near fork, with another plate with an apple and two bowls with produce, a canister and some metal objects.\nDecomposed sentence: [\"A counter top.\"]|[\"A plate with a fork and few scraps of food.\",\"A teddy bear lying with arm outstretched on plate near fork.\"]|[\"A plate with an apple.\"]|[\"Bowls with apples.\"]|[\"A canister.\"]|[\"Metal objects.\"]\nExample 9:\nOverall sentence: Indirect lighting in a darkened room plays across a table covered with the remains of a meal, thereby giving everything, including the table, a basket, plates with uneaten food, utensils and glasses with liquid, a reddish patina.\nDecomposed sentence: [\"Indirect lighting in a darkened room shines across a table covered with the remains of a meal.\",\"The table has a reddish patina.\",\"The basket has a reddish patina.\",\"The plates with uneaten food have a reddish patina.\",\"Utensils have a reddish patina.\",\"Glasses with liquid have a reddish patina.\"]\nExample 10:\nOverall sentence: the long view of a parking lot shows a row of parked vehicles, including a bus, which appears to be the destination of a number of similarly dressed women with luggage that are moving towards it from the immediate foreground.\nDecomposed sentence: [\"A parking lot shows a row of parked vehicles.\",\"A bus.\"]|[\"Similarly dressed women with luggages.\"]\nExample 11:\nOverall sentence: Corner view of a table with plaid cloth, red and white checkerboard napkins, a pizza, a container of green leafy vegetable, and some bottles, with the hips of a person wearing an apron standing beside table in green grass.\nDecomposed sentence: [\"Corner view of a table with plaid cloth.\",\"The hips of a person wearing an apron standing beside a table in green grass.\"]|[\"Red and white checkerboard napkins.\"]|[\"A pizza.\"]|[\"A container of green leafy vegetables.\"]|[\"Some bottles.\"]\nExample 12:\nOverall sentence: A room with natural, unfinished, pale wood walls and built in shelves, also has curtained sliding doors that open onto a patio with a lounge chair, a big double bed with a blue blanket, and various framed photos and books.\nDecomposed sentence: [\"A room.\"]|[\"Natural, unfinished, pale wood walls.\"]|[\"Built-in shelves.\"]|[\"Curtained sliding doors.\",\"Sliding doors that open onto a patio with a lounge chair.\"]|[\"A big double bed with a blue blanket.\"]|[\"Framed photos.\"]|[\"Books.\"]\nExample 13:\nOverall sentence: A minimally furnished room shows small window pairs with blinds and no curtains, an unmade double bed with a heavy, taupe duvet, and a white border near the ceiling with colorful birds trailing in both directions.\nDecomposed sentence: [\"A minimally furnished room.\"]|[\"Small window pairs with blinds and no curtains.\"]|[\"An unmade double bed with a heavy, taupe duvet.\"]|[\"A white border near the ceiling with colorful birds trailing in both directions.\"]\nExample 14:\nOverall sentence: A blurry night-time street scene shows a higher and lower roadway with overpasses and lots of traffic going the same way, while to the left an area of red glare with an arrow and number, suggests a caution area.\nDecomposed sentence: [\"A blurry night-time street scene shows a higher and lower roadway with overpasses and lots of traffic going the same way.\"]|[\"An area of red glare with an arrow and number, suggests a caution area.\"]\nExample 15:\nOverall sentence: Several people are standing, or sitting, in the vicinity of a scoreboard, to the front of which lies a tennis court, featuring a player with his knees bent, who is also grasping a racquet with both hands,\nDecomposed sentence: [\"Several people are standing, or sitting, in the vicinity of a scoreboard.\",\"In front of the scoreboard lies a tennis court.\"]|[\"A tennis player with his knees bent, who is grasping a racquet with both hands.\"]\nExample 16:\nOverall sentence: Black and white of a blacksmith interior with a horse, a man standing with arm out to it, and a man ducking under it's head and holding a long metal file in both hands, with horses on poles on the wall.\nDecomposed sentence: [\"Black and white photo of a blacksmith interior with a horse.\",\"Black and white photo of a man standing with arm out to a horse.\",\"Black and white photo of a man ducking under a horses head and holding a long metal file in both hands.\"]|[\"Black and white photo with horse poles on the wall.\"]\nExample 17:\nOverall sentence: A blurry living room scene suggests a wide screen T,V. with a game playing, bookshelves, a low cluttered table, a couch, and closest and clearest, a sports t-shirt with the number thirty-five on it.\nDecomposed sentence: [\"A blurry living room scene.\"]|[\"A wide screen TV with a sports game playing.\"]|[\"Bookshelves.\"]|[\"A low cluttered table.\"]|[\"A couch.\"]|[\"A sports t-shirt with the number 35 on it.\"]\nExample 18:\nOverall sentence: In the first picture a boy is flipping his skateboard, in the second picture is skating and lifting the board slightly off the ground, and in the third he is jumping into the air onto the skateboard.\nDecomposed sentence: [\"A boy is flipping his skateboard.\",\"A boy is skating and lifting the skateboard slightly off the ground.\",\"A boy is jumping into the air onto the skateboard.\"]\nExample 19:\nOverall sentence: A table with a person holding a sandwich in hand over a plate with a knife on it, and another plate with a knife, napkin, and french bread sandwich, along with a bottle and a glass with beverages.\nDecomposed sentence: [\"A table with a person holding a sandwich in hand.\",\"A sandwich in hand over a plate with a knife on it.\"]|[\"A plate with a knife, napkin, and french bread sandwich.\",\"A bottle.\",\"A glass with beverages.\"]\nExample 20:\nOverall sentence: An impressive array of motorcycles recedes into the distance between a driveway and a building fronted with cars and a grassy area, as lots of people mill about the driveway, around the bikes.\nDecomposed sentence: [\"An array of motorcycles between a driveway and a building.\",\"A building fronted with cars and a grassy area.\",\"Lots of people mill about the driveway, around the bikes.\"]\nIn the provided 20 examples, with overall sentence and their decomposed phrases, dependent phrases are grouped in the same set of brackets, while independent phrases are not grouped under the same set of brackets. Note that we define two sentences as dependent when the two phrases share an explicitly defined physical and material object between them in the overall sentence. Given an overall sentence and two decomposed phrases from user input, output whether the two decomposed phrases are dependent or independent. You may answer in only one digit, 1 meaning dependent, 0 meaning independent. Only output 1 if you are 100% certain. Do not consider any vague dependencies."},
          {"role": "user", "content": f"Overall sentence:{sentence}\n And two decomposed phrases: {segment1}, {segment2}\nPlease output 1 or 0, 1 meaning they are dependent phrases, 0 meaning they are independent phrases."}
    ]
    response = pipe(messages, **generation_args)
    result = response[0]['generated_text'].strip()
    #print(result)
    if result in {'1', '0'}:
      print(result)
      return int(result)
    else:
      print("non identified")
      return 0

def group_dependent_segments(sentence, segments):
    segments_list = segments.split('|')
    print(segments_list)
    n = len(segments_list)

    # Initialize Union-Find
    uf = UnionFind(n)

    # Perform pairwise comparisons sequentially
    for i in range(n):
        for j in range(i + 1, n):
            if uf.find(i) != uf.find(j):  # Check if i and j are already in the same group
                result = check_dependency(sentence, segments_list[i].strip(), segments_list[j].strip())
                if result == 1:
                    uf.union(i, j)
            else:
              print('already in same group')

    # Group segments based on union-find results
    groups = {}
    for i in range(n):
        root = uf.find(i)
        if root not in groups:
            groups[root] = []
        groups[root].append(i)

    # Convert indices back to segments
    grouped_indices = [",".join(map(str, sorted(group))) for group in groups.values()]
    return " | ".join(grouped_indices)

In [40]:
import time

In [47]:
def generate_decomp_and_dep(df_input):
  df = df_input
  df['caption_triples_ls'] = df['caption_mscoco'].apply(decompose_sentence)
  # Filter rows that do not start and end with "
  filtered_df = df[~(df['caption_triples_ls'].str.startswith('"') & df['caption_triples_ls'].str.endswith('"'))].copy()
  # Apply the lambda function to those rows only
  filtered_df['caption_triples_ls'] = filtered_df['caption_triples_ls'].apply(lambda x: '"' + x.split('"', 1)[-1].rsplit('"', 1)[0] + '"')
  # Combine the processed rows back with the original DataFrame
  result_df = pd.concat([df[(df['caption_triples_ls'].str.startswith('"') & df['caption_triples_ls'].str.endswith('"'))], filtered_df]).sort_index()
  result_df['groups'] = result_df.apply(lambda row: group_dependent_segments(row['caption_mscoco'], row['caption_triples_ls']), axis=1)
  return result_df

In [48]:
start_time = time.time()
result = generate_decomp_and_dep(df_mscoco[0:10])
end_time = time.time()
print(f"Time taken: {end_time - start_time} seconds")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


 "An institutional looking room."|"An exit sign."|"Chairs."|"Recording equipment."|"A man with one arm aloft."|"The man appears to be demonstrating something."|"A device in the held-up hand."|"For the benefit of a woman behind him."|"A woman is watching closely."
 "A black night."|"A blurry shot."|"A tall fence that angles in a kind of Z shape."|"Around what appears to be a parking area."|"A large building fronting the parking area."|"A kid with a red helmet."|"Takes a fierce swing in the batting area."|"Inside the fence."
 "An older looking model computer components."|"Two disk drive boxes stacked."|"A keyboard unit."|"A more modern looking computer monitor."|"Apple logo center bottom."|"On screen at top centered in light blue font letters is printed 'APPLE II'."
 "A motel-like room."|"The room features all of its furniture in rows."|"A sofa."|"A double bed."|"A stuffed chair."|"A desk chair."|"A desk."|"A bureau."|"A television."|"All of the furniture and one piece of artwork are in 

<ipython-input-47-d6fe5eee9cad>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['caption_triples_ls'] = df['caption_mscoco'].apply(decompose_sentence)
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


0
0
1
1
1
1
1
0
0
0
1
already in same group
already in same group
already in same group
0
1
already in same group
already in same group
already in same group
already in same group
1
already in same group
already in same group
already in same group
already in same group
already in same group
already in same group
already in same group
already in same group
already in same group
already in same group
already in same group
already in same group
already in same group
already in same group
['"A black night."', '"A blurry shot."', '"A tall fence that angles in a kind of Z shape."', '"Around what appears to be a parking area."', '"A large building fronting the parking area."', '"A kid with a red helmet."', '"Takes a fierce swing in the batting area."', '"Inside the fence."']
1
1
1
0
0
1
0
already in same group
already in same group
0
0
already in same group
1
already in same group
1
0
already in same group
already in same group
already in same group
1
already in same group
already in same gro

In [51]:
result

,id,image,caption_sharegpt4v,caption_mscoco,embedding,original_mscoco_caption_length,caption_triples_ls,groups
0,553992,000000553992.jpg,"In the image, a man and a woman are standing i...",An institutional looking room holds an exit si...,"[0.25389287, 0.2295701, -0.108173296, -0.07395...",249,"""An institutional looking room.""|""An exit sign...","0,1,2,3,4,5,6,7,8"
1,39270,000000039270.jpg,"In the image, there is a man who is in the mid...","Black night, a blurry shot shows a tall fence ...","[0.044150963, 0.17922473, -0.14461896, -0.1534...",246,"""A black night.""|""A blurry shot.""|""A tall fenc...","0,1,2,3,4,5,6,7"
2,277161,000000277161.jpg,"In the center of the image, a vintage Apple II...",An older looking model computer components; tw...,"[-0.41152698, -0.29223415, 0.025055759, -0.164...",244,"""An older looking model computer components.""|...","0,1,2,3,4,5"
3,380070,000000380070.jpg,"The image captures a cozy hotel room, bathed i...",A motel-like room features all of its furnitu...,"[0.23270163, 0.07637343, 0.36209187, -0.012398...",243,"""A motel-like room.""|""The room features all of...","0,1,2,3,4,5,6,7,8,9,10,11,12"
4,357272,000000357272.jpg,The image captures a room that exudes a vintag...,"A room shows a twin bed, crib, double bed, and...","[0.45787933, 0.13048139, 0.11703592, 0.0351443...",242,"""A room.""|""A twin bed.""|""A crib.""|""A double be...","0,1,2,3,4,5,6,7 | 8 | 9 | 10 | 11"
5,480770,000000480770.jpg,"In the heart of a bustling city, an elderly wo...",A street view shows a building entrance and tr...,"[0.20354465, -0.0718597, -0.27712327, -0.49626...",241,"""A street view.""|""A building entrance.""|""Traff...","0,1,2,3,4,5,6"
6,515241,000000515241.jpg,The image captures a dynamic moment in a baseb...,"Off in the distance, a few people dot the stan...","[-0.029054046, 0.23667967, 0.111569144, 0.0622...",235,"""Off in the distance, a few people dot the sta...","0,1,2,3,4,5"
7,560604,000000560604.jpg,"In the center of the image, a black stuffed an...",A counter top with a plate with a fork and few...,"[-0.29823345, 0.32871175, -0.07593347, -0.1097...",230,"""A counter top.""|""A plate with a fork and few ...","0,1,2 | 3 | 4 | 5 | 6"
8,158005,000000158005.jpg,"In the dimly lit ambiance of the room, a dinne...",Indirect lighting in a darkened room plays acr...,"[-0.18686089, 0.35955262, 0.008806132, -0.3130...",229,"""Indirect lighting in a darkened room.""|""Light...","0,1,2,3,4,5,6,7,8"
9,449712,000000449712.jpg,"In the image, a group of people are seen walki...",the long view of a parking lot shows a row of ...,"[0.19945762, -0.1157327, -0.03143066, 0.023830...",226,"""The long view of a parking lot.""|""Shows a row...","0,1,2,3,4,5"
